# Imports necessários

In [47]:
import numpy as np
import pandas as pd
import pyodbc
from dotenv import load_dotenv
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Conexão e query's

### Conexão com o DB

In [3]:
def connect_db():
    load_dotenv()  # carrega as variáveis de ambiente do arquivo .env

    server = os.getenv('DB_SERVER')
    database = os.getenv('DB_DATABASE')
    username = os.getenv('DB_USERNAME')
    password = os.getenv('DB_PASSWORD')

    global cnxn
    if 'cnxn' in globals():
        try:
            cnxn.close()
            print('Conexão anterior fechada com sucesso.')
            return None
        except Exception as e:
            print(f'Não foi possível fechar a conexão anterior. Erro: {e}')

    try:
        cnxn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}")
        print('Conexão bem sucedida.')
        return cnxn
    except Exception as err:
        print(f'Conexão mal sucedida. Erro {err}')
        return None

### Querys de busca no DB

In [4]:
query_get_data_receitas = 'SELECT * FROM dbo.Fato_Receita'

# Import dos dados constantes no DW

## Dados sobre as Receitas

In [5]:
df_receitas = pd.read_sql(query_get_data_receitas, connect_db())
connect_db()
df_receitas

Conexão bem sucedida.


C:\Users\allys\AppData\Local\Temp\ipykernel_17920\1582883830.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_receitas = pd.read_sql(query_get_data_receitas, connect_db())


Conexão anterior fechada com sucesso.


,id,uid_fato_receita,cod_receita,data_fato,contribuinte_receita,valor
0,1,229F9CDC-2494-4933-B43E-9ADD784F01B1,1722010101,2013-01-03,886cd0eabf5a18,185560.68
1,2,B8D6B25C-8928-4887-B189-0D9B5D17E11D,1722010101,2013-01-04,886cd0eabf5a18,8081.80
2,3,F63D84B9-0BF1-46AD-B381-AF189BF428BF,1722010101,2013-01-07,886cd0eabf5a18,4231.39
3,4,F645121D-AA82-4EDB-8A86-7F41154D65AF,1722010101,2013-01-08,886cd0eabf5a18,10536.53
4,5,28DE0FD1-6032-4A2C-B16F-F250E9B31DD1,1722010101,2013-01-15,886cd0eabf5a18,103118.27
...,...,...,...,...,...,...
102288,102289,EF8465D6-48C6-44C8-9741-FEB45515F37B,191101011,2022-01-19,e6da541375c18b,4374.24
102289,102290,DEDC9308-3ED5-498A-A904-E9551AC67411,112101011,2022-01-18,e6da541375c18b,228.00
102290,102291,1DDFCD5B-3F8A-4C08-B899-4331C5011C02,191101011,2022-01-18,e6da541375c18b,3758.69
102291,102292,CC42EB8D-FF4C-4126-81A9-01FE36DF882D,112101011,2022-01-17,e6da541375c18b,4297.00


In [31]:
df_receitas_copy = df_receitas[['cod_receita','data_fato','contribuinte_receita','valor']].copy()
df_receitas_copy['ano'] = df_receitas_copy['data_fato'].dt.year
df_receitas_copy['mes'] = df_receitas_copy['data_fato'].dt.month
df_receitas_copy['dia'] = df_receitas_copy['data_fato'].dt.day
df_receitas_copy

,cod_receita,data_fato,contribuinte_receita,valor,ano,mes,dia
0,1722010101,2013-01-03,886cd0eabf5a18,185560.68,2013,1,3
1,1722010101,2013-01-04,886cd0eabf5a18,8081.80,2013,1,4
2,1722010101,2013-01-07,886cd0eabf5a18,4231.39,2013,1,7
3,1722010101,2013-01-08,886cd0eabf5a18,10536.53,2013,1,8
4,1722010101,2013-01-15,886cd0eabf5a18,103118.27,2013,1,15
...,...,...,...,...,...,...,...
102288,191101011,2022-01-19,e6da541375c18b,4374.24,2022,1,19
102289,112101011,2022-01-18,e6da541375c18b,228.00,2022,1,18
102290,191101011,2022-01-18,e6da541375c18b,3758.69,2022,1,18
102291,112101011,2022-01-17,e6da541375c18b,4297.00,2022,1,17


## Análise regressiva

#### Definindo as variáveis independentes (y) e variável dependente(x)

In [50]:
X = df_receitas_copy['valor']
y = df_receitas_copy['cod_receita']

#### Dividindo o conjunto de dados em conjutos de treinamento e teste

#### Divisão dos dados em conjuntos de treinamento e teste

Neste trecho de código, estamos usando a função `train_test_split` da biblioteca Scikit-Learn para dividir o conjunto de dados em conjuntos de treinamento e teste. A função recebe como argumentos as variáveis independentes (X) e a variável dependente (y), bem como outros parâmetros opcionais.

Os parâmetros `test_size` e `random_state` são definidos na chamada da função e afetam a maneira como os dados são divididos em conjuntos de treinamento e teste. 

- `test_size`: define a proporção dos dados que serão utilizados para teste. Nesse caso, `test_size=0.2` significa que 20% dos dados serão usados para teste e 80% para treinamento.

- `random_state`: define uma semente aleatória para garantir que a divisão dos dados seja sempre a mesma, facilitando a reprodução dos resultados. Nesse caso, `random_state=42` é uma escolha comum, mas poderia ser qualquer outro valor.

A função `train_test_split` retorna quatro conjuntos de dados:

- `X_train`: conjunto de dados de treinamento para as variáveis independentes.
- `X_test`: conjunto de dados de teste para as variáveis independentes.
- `y_train`: conjunto de dados de treinamento para a variável dependente.
- `y_test`: conjunto de dados de teste para a variável dependente.

Esses conjuntos de dados são usados para treinar e testar o modelo de regressão. O conjunto de treinamento é usado para ajustar o modelo aos dados, enquanto o conjunto de teste é usado para avaliar o desempenho do modelo em dados novos e não vistos anteriormente.

É importante dividir os dados em conjuntos de treinamento e teste para evitar o overfitting, que ocorre quando o modelo é ajustado aos dados de treinamento de forma tão precisa que se torna incapaz de generalizar para novos dados. A divisão dos dados permite avaliar o desempenho do modelo em dados não utilizados para treinamento e ajustar os parâmetros do modelo para melhorar sua capacidade de generalização.


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Instanciando o modelo utilizado

#### Regressão Linear

In [52]:
# Transformando X_train e X_test em um array numpy bidimensional
X_train = X_train.values.reshape((-1, 1))
X_test = X_test.values.reshape((-1, 1))

In [53]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

#### Utilizando o modelo para previsões nos dados de teste

In [54]:
y_pred = model.predict(X_test)

#### Avaliando o desempenho com coef. determinação (r2)

In [55]:
r2_score(y_test, y_pred)

0.0012093665558950972

In [56]:
model.score(X_test, y_test)

0.0012093665558950972